### Using Train and Test Set 3
#### Pre-Processing of Amazon text is already completed (check other notebook)
#### Converted from multi-class to binary output
#### Removed duplicate reivews
#### Eliminated stop words
#### Lower case all words
#### Nulls removed
#### Lemmetization done

In [31]:
import pandas as pd
import numpy as np

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
from google.colab import drive # mount drive 
drive.mount('/home/gdrive')

Mounted at /home/gdrive


### Training Data

In [34]:
reviews_train = pd.read_csv('/home/gdrive/My Drive/train_set3.csv')

In [35]:
reviews_train.isna().sum()

review    0
label     0
dtype: int64

In [36]:
#reviews_train.dropna(inplace = True)

In [37]:
#reviews_train.isna().sum()

In [38]:
reviews_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91185 entries, 0 to 91184
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  91185 non-null  object
 1   label   91185 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [39]:
reviews_train.head()  # total test records = 91185

,review,label
0,great product kid love good cooking buy store ...,1
1,wonderful texture great taste thank mr wei hel...,1
2,really work mask pill seen dog yet like either...,1
3,think thin changed packaging big deal unfortun...,0
4,worst coffee beverage ever grove square also c...,0


### Test Data

In [40]:
reviews_test = pd.read_csv('/home/gdrive/My Drive/test_set3.csv')

In [41]:
reviews_test.isna().sum()

review    0
label     0
dtype: int64

In [42]:
#reviews_test.dropna(inplace = True)

In [43]:
#reviews_test.isna().sum()

In [44]:
reviews_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22797 entries, 0 to 22796
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  22797 non-null  object
 1   label   22797 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 356.3+ KB


In [45]:
reviews_test.head() # total test records = 22797

,review,label
0,received month ice cream club gift disappointe...,0
1,best year whether need also make good christma...,1
2,description give specification item would grea...,0
3,excellent flavor great choice looking taste ov...,1
4,read review purchased looking forward lasting ...,0


### Combine Both Training and Testing Datasets into one Corpus of text data and then converting the text into vector

In [46]:
corpus = []

for i in range(len(reviews_train['review'])):
  review = reviews_train['review'][i]
  corpus.append(review)
print("Number of reviews in trained dataset", len(corpus)) # only training data first added to corpus
num_train = len(corpus) # length of training data


for i in range(len(reviews_test['review'])):
  review = reviews_test['review'][i]
  corpus.append(review)

print("Number of reviews in training dataset", len(corpus)) # combines training + test data (total data) to corpus
num_test = len(corpus) - num_train # length of test data

print(len(corpus))

# Therefore: index 0-91185 (training data) & index 91186-113982 (test data)

Number of reviews in trained dataset 91185
Number of reviews in training dataset 113982
113982


### Create Word2Vec Matrix

In [47]:
from gensim.models import Word2Vec

In [48]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 4    # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

#size = 300
# Initialize and train the model only using training data (final_string)

w2v_model = Word2Vec(sentences = corpus,size= 300)

In [49]:
# get the shape of the word2vec embedding matrix
w2v_model.syn1neg.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  


(27, 300)

### Tokenize the whole corpus data

In [50]:
# USING KERAS TOOLS FOR CONVERTING TEXTS TO FORMATS: RUN a Tokenizer using the top 12000 words as features, POST-PREPROCESSING OF TEXT
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MAX_FEATURES = 12000
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(corpus)

vocab_size = len(tokenizer.word_index) + 1

tokened_corpus = tokenizer.texts_to_sequences(corpus)

In [51]:
print(tokened_corpus[0]) 
print(len(tokened_corpus[0]))

print(vocab_size)

[10, 3, 221, 12, 6, 428, 20, 37, 219, 188, 461, 103, 103, 6, 289]
15
60324


### Create Embedding Matrix for Corpus Using Word2Vec

https://www.programcreek.com/python/?CodeExample=get%20embedding%20matrix

https://jjallaire.github.io/deep-learning-with-r-notebooks/notebooks/6.1-using-word-embeddings.nb.html

https://gdcoder.com/what-is-an-embedding-layer/

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

In [60]:
# create a embedding matrix for words that are in our combined train and test data
from numpy import zeros
from gensim.models import KeyedVectors

embedding_matrix = zeros((vocab_size, 300)) # embed_size = 300 too small so use 1500 instead?

for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_vector = w2v_model.wv[word] 
  #if word in w2v_model.wv:      # check if the word is in the word2vec vocab
    #embedding_vector = w2v_model.wv[word].get(word)# if yes, then add to embedding vector
  if embedding_vector is not None: # word is in vocav learned by word2vec model
    embedding_matrix[i] = embedding_vector #if word is not found then embedding vector corresponding to that vector will stay zero

In [61]:
# check the shape of the new embedding matrix
embedding_matrix.shape

(60324, 300)

In [62]:
print(embedding_vector)
print(len(embedding_vector))

[-5.46189137e-02 -4.68912162e-02  5.57293929e-02 -3.13567549e-01
 -1.95936203e-01  1.81713864e-01 -2.09030151e-01 -4.34218235e-02
  2.92321920e-01  1.69093497e-02 -1.44047499e-01  2.22198237e-02
 -1.62296612e-02  3.50672364e-01 -5.10755219e-02 -1.66118070e-01
 -1.46687701e-01  3.50952474e-03  5.36353253e-02  2.71287956e-03
 -3.12871158e-01 -2.10546359e-01  4.19280641e-02 -1.58995565e-03
 -3.28889340e-01 -2.21481606e-01 -6.34755641e-02  1.79600284e-01
 -1.87404618e-01  5.35867959e-02 -5.29254824e-02 -4.00898941e-02
 -1.97423846e-01  3.08969736e-01  1.63269907e-01 -1.60739690e-01
 -4.66797203e-01  3.12889159e-01 -1.26704156e-01  2.41227210e-01
 -1.16399415e-01  2.71447569e-01 -8.69302899e-02  5.58016263e-02
  3.16206515e-02  8.14128146e-02 -7.03214407e-02  1.98231384e-01
  1.42275468e-01  4.31462526e-02 -1.04637578e-01 -3.02011762e-02
 -7.63651058e-02  7.11790845e-02  1.47278577e-01 -1.70239527e-02
  2.72472262e-01  2.27570295e-01  3.31337541e-01 -1.64590999e-01
 -7.22527504e-02 -2.01405

In [63]:
print(embedding_matrix)
print(len(embedding_matrix))

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.05461891 -0.04689122  0.05572939 ...  0.10106676  0.05168538
   0.08977326]
 [-0.05461891 -0.04689122  0.05572939 ...  0.10106676  0.05168538
   0.08977326]
 ...
 [-0.05461891 -0.04689122  0.05572939 ...  0.10106676  0.05168538
   0.08977326]
 [-0.05461891 -0.04689122  0.05572939 ...  0.10106676  0.05168538
   0.08977326]
 [-0.05461891 -0.04689122  0.05572939 ...  0.10106676  0.05168538
   0.08977326]]
60324


In [56]:
# Tell me what words are most similar to the word 'good'?
#w2v_model.most_similar('good')

In [57]:
#This is how to access the embedding vector for a given word

#w2v_model.wv['awesome']

### Padding Sequences

In [64]:
# PADDING SEQUENCES
#In order to use batches effectively, we need to take these sequences and turn them into sequences of the same length. 
# In this case we can make everything here the length of the longest sentence in the training set. 

# Padd whole corpus data
MAX_LENGTH = max(len(train_ex) for train_ex in tokened_corpus)

print(MAX_LENGTH) # max sequence length
corpus_padded = pad_sequences(tokened_corpus, maxlen=MAX_LENGTH)


# Split word embedded padded data into train and test
X_train_padded = corpus_padded[:num_train] #index 0-91185 (training data)
X_test_padded = corpus_padded[num_train:] #index 91186-113982 (test data)

# Therefore: index 0-91185 (training data) & index 91186-113982 (test data)

print("Number of reviews in train data after padding sequences: ",len(X_train_padded))
print("Number of reviews in test data after padding sequences: ",len(X_test_padded))

# Prepare labels for train and test also
y_train = reviews_train['label']
y_test = reviews_test['label']

print("Number of labels in train data: ",len(y_train))
print("Number of labels in test data: ",len(y_test))



1442
Number of reviews in train data after padding sequences:  91185
Number of reviews in test data after padding sequences:  22797
Number of labels in train data:  91185
Number of labels in test data:  22797


### CNN Model Using W2V Word Embedding

In [91]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Flatten, Conv1D, MaxPooling1D, GlobalMaxPool1D, Activation
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.python.keras import models, layers, optimizers
import tensorflow

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

# input_dim = vocab_size = 60324
# input_length = MAX_LENGTH = 1442
# MAX_FEATURES = 12000
# output_dim = 300

In [92]:
def build_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(input_dim = vocab_size, output_dim = 300, weights=[embedding_matrix], input_length=MAX_LENGTH, trainable = True)(sequences)
    x = layers.Conv1D(64, 3, activation='relu')(embedded)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(3)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool1D(5)(x)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model
    
model = build_model()

In [93]:
model.fit(X_train_padded, y_train, batch_size=128, epochs=5, validation_split = 0.2)

Epoch 1/5
570/570 [==============================] - 180s 316ms/step - loss: 0.5435 - accuracy: 0.7014 - val_loss: 0.4087 - val_accuracy: 0.8213
Epoch 2/5
570/570 [==============================] - 177s 310ms/step - loss: 0.3129 - accuracy: 0.8666 - val_loss: 0.3402 - val_accuracy: 0.8564
Epoch 3/5
570/570 [==============================] - 178s 312ms/step - loss: 0.2595 - accuracy: 0.8939 - val_loss: 0.3390 - val_accuracy: 0.8557
Epoch 4/5
570/570 [==============================] - 178s 312ms/step - loss: 0.2156 - accuracy: 0.9157 - val_loss: 0.3632 - val_accuracy: 0.8543
Epoch 5/5
570/570 [==============================] - 177s 310ms/step - loss: 0.1629 - accuracy: 0.9371 - val_loss: 0.4081 - val_accuracy: 0.8498


In [96]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

preds = model.predict(X_test_padded)
print('Accuracy score: {:0.4}'.format(accuracy_score(y_test, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(y_test, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(y_test, preds)))

Accuracy score: 0.8485
F1 score: 0.8432
ROC AUC score: 0.923


In [97]:
# evaluate
loss, acc = model.evaluate(X_test_padded, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 84.848881
